In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn import preprocessing
from sklearn.model_selection import LeavePOut
from sklearn import decomposition
from sklearn import model_selection

from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
pd.set_option('display.max_columns', 100)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


data = pd.read_csv('C:/Users/u0132612/Documents/PhD KULeuven/Lessen/Advanced Analytics in a Big Data World/Assignments/Assignment 2/train.csv')
data_test = pd.read_csv('C:/Users/u0132612/Documents/PhD KULeuven/Lessen/Advanced Analytics in a Big Data World/Assignments/Assignment 2/test.csv')

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 22 columns):
id                     510 non-null int64
name                   510 non-null object
brand                  510 non-null object
base_name              510 non-null object
screen_size            510 non-null float64
pixels_x               510 non-null int64
pixels_y               510 non-null int64
screen_surface         498 non-null object
touchscreen            510 non-null int64
cpu                    510 non-null object
cpu_details            504 non-null object
detachable_keyboard    506 non-null float64
discrete_gpu           510 non-null int64
gpu                    508 non-null object
os                     508 non-null object
os_details             508 non-null object
ram                    510 non-null int64
ssd                    510 non-null int64
storage                510 non-null int64
weight                 506 non-null float64
min_price              510 non-null floa

In [16]:

#Same preprocessing as Script Steven

data.pixels_y = data.pixels_y.astype('int64')

data_test.pixels_y = data_test.pixels_y.fillna(0)
data_test.pixels_y = data_test.pixels_y.astype('int64')

#selecteer enkel usefull information
data_s = data[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', "cpu_details", 'discrete_gpu','gpu','os','os_details','ram','ssd','storage','weight','min_price','max_price']]
data_test_s = data_test[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', 'cpu_details', 'discrete_gpu','gpu',"os",'os_details','ram','ssd','storage','weight']]

#first basic reformatter
def reformat(df):
    df_pp = df.copy()
    df_pp["cpu_brand"] = df_pp.cpu.str.split(n=1).str[0]
    df_pp["cpu_type"] = df_pp.cpu.str.split(n=1).str[1]
    df_pp["gpu_brand"] = df_pp.gpu.str.split(n=1).str[0]
    df_pp["gpu_series"] = df_pp.gpu.str.split(n=2).str[1]
    df_pp["os_type"] = df_pp.os_details.str.split(n=1).str[1]
    df_pp["cpu_gen"] = df_pp['cpu_details'].str.split("(").str[1].str.split(")").str[0].str.split(" ",n=3).str[0]
    df_pp = df_pp.drop(columns=["os_details", "cpu", "gpu", "os_details"])
    return df_pp

#automatische onehot encoder
def process_onehot(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        enc = preprocessing.OneHotEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        new_df = pd.DataFrame(data=enc.transform(df[[item]].dropna()).toarray(), columns=enc.categories_[0])
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

#automatische ordinal encoder
def process_ordinal(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        new_df = pd.DataFrame()
        enc = preprocessing.OrdinalEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        temp = enc.transform(df[[item]].dropna()).shape
        new_df[item] = enc.transform(df[[item]].dropna()).flatten()
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

data_s = reformat(data_s)
data_test_s = reformat(data_test_s)

data_t = pd.concat([data_s, data_test_s]).reset_index(drop=True)

processed_data_total = process_ordinal(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand', "cpu_gen","os" ])

data_t = data_t[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]

data_total = pd.concat([processed_data_total, data_t], axis=1).set_index("id")

#selecteer de juiste datasets -> training voor model training en test de test dataset van seppe

data_total_train = data_total.filter(items=data.id, axis=0)
data_total_test = data_total.filter(items=data_test.id, axis=0).iloc[:,0:-2].fillna(0)
index=data_total_test.index

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [17]:
X_1 = data_total_train.iloc[:,0:-2].fillna(0)
y_min = data_total_train.iloc[:,-2].fillna(0)
X_2 = data_total_train.iloc[:,0:-2].fillna(0)
y_max = data_total_train.iloc[:,-1].fillna(0)


In [18]:
pp = preprocessing.PowerTransformer(standardize=True)


In [27]:
# Build XGBoost model

    # MINIMUM price model
    
MinDmatrix= xgb.DMatrix(X_1, label=y_min)

X_train, X_test, y_train, y_test = train_test_split(X_1,y_min,test_size=.3, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

from sklearn.metrics import mean_absolute_error

# Calculate a simple baseline based on mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))

# Develop more advanced model with Tuning

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

params['eval_metric'] = "mae"
num_boost_round = 999                  #I will apply early stopping later on

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

print("After the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. This is the MAE of our model with default parameters and an optimal number of boosting rounds, on the test dataset. As you can see, we are already beating the baseline")

Baseline MAE is 448.74
[13:32:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:596.06610
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:432.23444
[2]	Test-mae:319.81326
[3]	Test-mae:256.49149
[4]	Test-mae:221.46075
[5]	Test-mae:198.40144
[6]	Test-mae:189.22231
[7]	Test-mae:181.88353
[8]	Test-mae:178.59573
[9]	Test-mae:177.80931
[10]	Test-mae:178.17625
[11]	Test-mae:176.70717
[12]	Test-mae:177.37080
[13]	Test-mae:178.18309
[14]	Test-mae:179.28029
[15]	Test-mae:180.40326
[16]	Test-mae:181.15660
[17]	Test-mae:181.37364
[18]	Test-mae:181.55345
[19]	Test-mae:181.89458
[20]	Test-mae:181.77472
[21]	Test-mae:181.63690
Stopping. Best iteration:
[11]	Test-mae:176.70717

Best MAE: 176.71 with 12 rounds
After the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. This is the MAE of our model with default para

In [31]:
#In order to tune the other hyperparameters, we will use the cv function from XGBoost. 
#It allows to run cross-validation on the training dataset and returns a mean MAE score.


cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

cv_results['test-mae-mean'].min()

# Start with max depth and min child weight (maybe should do this in a GridSearch that combines all hyperparms)

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,15)
    for min_child_weight in range(1,25)
]

min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))


In order to tune the other hyperparameters, we will use the cv function from XGBoost. It allows us to run cross-validation on our training dataset and returns a mean MAE score.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:16

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


	MAE 161.4723816 for 29 rounds
CV with max_depth=2, min_child_weight=2
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 158.5092316 for 38 rounds
CV with max_d

In [33]:
# Did exagerate a bit but best params are max_depth=5 & min_child_weight=6 [still model for min value]

params['max_depth'] = 5
params['min_child_weight'] = 6

# Let's look for subsample and colsample optimal values

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 152.267633 for 11 rounds
CV with subsample=1.0, colsample=0.9
[13:52:27] 

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


	MAE 151.75036899999998 for 14 rounds
CV with subsample=1.0, colsample=0.8
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 152.509549 for 8 rounds
CV with sub

In [50]:
# best params are subsample=0.8 & colsample= 0.7 [still model for min value]

params['subsample'] = .8
params['colsample_bytree'] = 0.7

# Now i look for the optimal learning rate (eta)

min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))


CV with eta=0.3
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 145.8399596 for 8 rounds

CV with eta=0.2
[14:05:29] WARNING: C:/Users/Administrator/workspace

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


	MAE 145.8667572 for 39 rounds

CV with eta=0.05
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:05:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 145.97923899999998 for 71 rounds

CV with eta=0.01
[14:05:29]

In [53]:
# best param for learning rate = 0.01 [still model for min value]

params['eta'] = .01

# Create param dict with the optimal hyperparams

params
{'colsample_bytree': 0.7,
 'eta': 0.01,
 'eval_metric': 'mae',
 'max_depth': 5,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 0.8}

# Train model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1)) 

# Best MAE: 155.62 in 385 rounds

[14:13:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:820.57416
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:812.89685
[2]	Test-mae:805.27716
[3]	Test-mae:797.43518
[4]	Test-mae:789.87116
[5]	Test-mae:782.53925
[6]	Test-mae:775.37250
[7]	Test-mae:768.05658
[8]	Test-mae:760.85828
[9]	Test-mae:753.79022
[10]	Test-mae:746.90283
[11]	Test-mae:739.60608
[12]	Test-mae:732.75977
[13]	Test-mae:725.64648
[14]	Test-mae:718.73138
[15]	Test-mae:712.16754
[16]	Test-mae:705.58026
[17]	Test-mae:698.81598
[18]	Test-mae:692.04364
[19]	Test-mae:685.57404
[20]	Test-mae:679.15637
[21]	Test-mae:672.81982
[22]	Test-mae:666.46844
[23]	Test-mae:659.88184
[24]	Test-mae:653.76178
[25]	Test-mae:647.70789
[26]	Test-mae:641.63910
[27]	Test-mae:635.65796
[28]	Test-mae:629.73291
[29]	Test-mae:623.81799
[30]	Test-mae:618.15326
[31]	Test-mae:612.34357
[32]	Test

In [56]:
num_boost_round = model.best_iteration + 1

# Before using it for predictions, we should retrain it with the good number of rounds. 
#Since we know the exact best num_boost_round, we don't need the early_stopping_round anymore.

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

# Make predictions with the model on test data set

mean_absolute_error(best_model.predict(dtest), y_test) # MAE = 155.62180246789472 in 385 rounds

[14:19:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:820.57416
[1]	Test-mae:812.89685
[2]	Test-mae:805.27716
[3]	Test-mae:797.43518
[4]	Test-mae:789.87116
[5]	Test-mae:782.53925
[6]	Test-mae:775.37250
[7]	Test-mae:768.05658
[8]	Test-mae:760.85828
[9]	Test-mae:753.79022
[10]	Test-mae:746.90283
[11]	Test-mae:739.60608
[12]	Test-mae:732.75977
[13]	Test-mae:725.64648
[14]	Test-mae:718.73138
[15]	Test-mae:712.16754
[16]	Test-mae:705.58026
[17]	Test-mae:698.81598
[18]	Test-mae:692.04364
[19]	Test-mae:685.57404
[20]	Test-mae:679.15637
[21]	Test-mae:672.81982
[22]	Test-mae:666.46844
[23]	Test-mae:659.88184
[24]	Test-mae:653.76178
[25]	Test-mae:647.70789
[26]	Test-mae:641.63910
[27]	Test-mae:635.65796
[28]	Test-mae:629.73291
[29]	Test-mae:623.81799
[30]	Test-mae:618.15326
[31]	Test-mae:612.34357
[32]	Test-mae:606.52887
[33]	Test-mae:600.92249
[34]	Test-mae:595

155.62180246789472

In [60]:
# Build XGBoost model

    # MAXIMUM price model
    
MaxDmatrix= xgb.DMatrix(X_2, label=y_max)

X_train, X_test, y_train, y_test = train_test_split(X_2,y_max,test_size=.3, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

from sklearn.metrics import mean_absolute_error

# Calculate a simple baseline based on mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE MAX is {:.2f}".format(mae_baseline))

# Develop more advanced model with Tuning

params = {
    'max_depth':6,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective':'reg:linear',
}

params['eval_metric'] = "mae"
num_boost_round = 999                  #I will apply early stopping later on

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE MAX: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

print("Again, after the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. This is the MAE of our model with default parameters and an optimal number of boosting rounds, on the test dataset.")




Baseline MAE MAX is 465.21
[14:47:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:617.53790
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:444.47839
[2]	Test-mae:322.49149
[3]	Test-mae:265.82123
[4]	Test-mae:229.68079
[5]	Test-mae:206.66199
[6]	Test-mae:193.92419
[7]	Test-mae:188.03079
[8]	Test-mae:185.18602
[9]	Test-mae:184.32861
[10]	Test-mae:183.10942
[11]	Test-mae:182.82877
[12]	Test-mae:183.69342
[13]	Test-mae:183.55371
[14]	Test-mae:184.71942
[15]	Test-mae:185.59464
[16]	Test-mae:186.00621
[17]	Test-mae:184.85701
[18]	Test-mae:185.43532
[19]	Test-mae:185.26804
[20]	Test-mae:185.68230
[21]	Test-mae:186.39064
Stopping. Best iteration:
[11]	Test-mae:182.82877

Best MAE MAX: 182.83 with 12 rounds
After the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. This is the MAE of our model with defa

In [61]:

# Tune other hyperparams, using cv function from XGBoost. Run cross-val on training and return mean MAE score

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

cv_results['test-mae-mean'].min()

# Start with max depth and min child weight (maybe should do this in a GridSearch that combines all hyperparms)

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,15)
    for min_child_weight in range(1,25)
]

min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))


[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
CV with max_depth=2, min_child_weight=1
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.



	MAE 157.5360074 for 39 rounds
CV with max_depth=2, min_child_weight=3
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:52:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 155.63081359999998 for 42 rounds
CV wi

In [62]:
# Best params are max_depth= 5 & min_child_weight= 7 [still model for Max price]

params['max_depth'] = 5
params['min_child_weight'] = 7

# Let's look for subsample and colsample optimal values

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 152.5074432 for 9 rounds
CV with subsample=1.0, colsample=0.9
[14:56:58] 

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.



CV with subsample=1.0, colsample=0.7
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:56:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 154.0374632 for 9 rounds
CV with subsample=1.0, colsample=0.6
[14:56:58]

In [63]:
# best params are subsample=0.9 & colsample= 1.0 [still model for MAX value]

params['subsample'] = .9
params['colsample_bytree'] = 1.0

# Now i look for the optimal learning rate (eta)

min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 151.77083720000002 for 12 rounds

CV with eta=0.2
[14:59:18] WARNING: C:/Users/Administrator/w

C:\Users\u0132612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


	MAE 155.67492059999998 for 41 rounds

CV with eta=0.05
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[14:59:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 153.10381180000002 for 70 rounds

CV with eta=0.01
[14

In [64]:
# best param for learning rate = 0.3 (really different from min model, 0.01) [still model for MAX value]

params['eta'] = .3

# Create param dict with the optimal hyperparams

params
{'colsample_bytree': 1.0,
 'eta': 0.3,
 'eval_metric': 'mae',
 'max_depth': 5,
 'min_child_weight': 7,
 'objective': 'reg:linear',
 'subsample': 0.9}

# Train model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE MAX: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1)) 

# Best MAE MAX: 159.17 in 28 rounds (<->Best MAE min was 155.62 in 385! rounds)

[15:01:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:618.87463
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:443.90079
[2]	Test-mae:331.42105
[3]	Test-mae:258.18955
[4]	Test-mae:219.32021
[5]	Test-mae:198.46870
[6]	Test-mae:183.57134
[7]	Test-mae:177.19913
[8]	Test-mae:175.47463
[9]	Test-mae:174.20999
[10]	Test-mae:173.39416
[11]	Test-mae:173.27182
[12]	Test-mae:169.30847
[13]	Test-mae:168.69304
[14]	Test-mae:167.58795
[15]	Test-mae:166.79279
[16]	Test-mae:166.09082
[17]	Test-mae:165.59865
[18]	Test-mae:164.96849
[19]	Test-mae:164.16205
[20]	Test-mae:164.46808
[21]	Test-mae:164.78838
[22]	Test-mae:164.04040
[23]	Test-mae:162.58511
[24]	Test-mae:162.21074
[25]	Test-mae:160.92313
[26]	Test-mae:160.03763
[27]	Test-mae:159.16917
[28]	Test-mae:160.48439
[29]	Test-mae:160.29446
[30]	Test-mae:160.14348
[31]	Test-mae:160.20891
[32]	Test

In [65]:
num_boost_round = model.best_iteration + 1

# Before using it for predictions, we should retrain it with the good number of rounds. 
#Since we know the exact best num_boost_round, we don't need the early_stopping_round anymore.

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

# Make predictions with the model on test data set (VALIDATION SET based on training set)

mean_absolute_error(best_model.predict(dtest), y_test) # MAE max= 159.16917813968035 in 28 rounds (MAE Min = 155.62180246789472 in 385 rounds)

[15:04:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:618.87463
[1]	Test-mae:443.90079
[2]	Test-mae:331.42105
[3]	Test-mae:258.18955
[4]	Test-mae:219.32021
[5]	Test-mae:198.46870
[6]	Test-mae:183.57134
[7]	Test-mae:177.19913
[8]	Test-mae:175.47463
[9]	Test-mae:174.20999
[10]	Test-mae:173.39416
[11]	Test-mae:173.27182
[12]	Test-mae:169.30847
[13]	Test-mae:168.69304
[14]	Test-mae:167.58795
[15]	Test-mae:166.79279
[16]	Test-mae:166.09082
[17]	Test-mae:165.59865
[18]	Test-mae:164.96849
[19]	Test-mae:164.16205
[20]	Test-mae:164.46808
[21]	Test-mae:164.78838
[22]	Test-mae:164.04040
[23]	Test-mae:162.58511
[24]	Test-mae:162.21074
[25]	Test-mae:160.92313
[26]	Test-mae:160.03763
[27]	Test-mae:159.16917


159.16917813968035

In [74]:
#total MAE based on created VALIDATION set BUT averaged over the PC ID's
#the test set was not used

MAEmin = 155.62180246789472
MAEmax = 159.16917813968035

totalerror = MAEmin + MAEmax
print(totalerror)


314.7909806075751
